In [1]:
import pandas as pd
import numpy as np
from utils import *
from data_processing import *

# load general data
general_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\general_data_champions-tour-2024-pacific-kickoff.csv')
general_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\general_data_champions-tour-2024-americas-kickoff.csv')
general_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\general_data_champions-tour-2024-emea-kickoff.csv')

# performance data
performance_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\performance_data_champions-tour-2024-pacific-kickoff.csv')
performance_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\performance_data_champions-tour-2024-americas-kickoff.csv')
performance_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\performance_data_champions-tour-2024-emea-kickoff.csv')

# economic data
economy_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\economy_data_champions-tour-2024-pacific-kickoff.csv')
economy_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\economy_data_champions-tour-2024-americas-kickoff.csv')
economy_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\economy_data_champions-tour-2024-emea-kickoff.csv')

# picks and bans
pick_ban_pacific = pd.read_csv('champions-tour-2024-pacific-kickoff_data\pick_ban_data_champions-tour-2024-pacific-kickoff.csv')
pick_ban_americas = pd.read_csv('champions-tour-2024-americas-kickoff_data\pick_ban_data_champions-tour-2024-americas-kickoff.csv')
pick_ban_emea = pd.read_csv('champions-tour-2024-emea-kickoff_data\pick_ban_data_champions-tour-2024-emea-kickoff.csv')

2024-05-10 17:00:55.491 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-10 17:00:55.494 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-10 17:00:55.495 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-10 17:00:55.496 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-10 17:00:55.498 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-10 17:00:55.499 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-05-10 17:00:55.501 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [39]:
df_name_team = pd.read_csv('data/team_names.csv')
data_dict = df_name_team.to_dict(orient='records')
# general_emea['Team Name'].apply(lambda x: df_name_team[df_name_team['FullName'] == x]['SurName'])

In [36]:
set(pick_ban_emea['Team Name'])

{'BBL', 'FNC', 'FUT', 'GX', 'KC', 'KOI', 'M8', 'NAVI', 'TH', 'TL', 'VIT'}

<h1>Individual Data</h1>
<h2>General Data</h2>

In [2]:
df_pacific_general = general_feature_creation_for_teams(general_pacific)
df_americas_general = general_feature_creation_for_teams(general_americas)
df_emea_general = general_feature_creation_for_teams(general_emea)

df_general = create_dataframe(df_emea_general,df_americas_general,df_pacific_general)
# Assuming the last column of the data is the target variable (e.g., region labels)
X_general, y_general = df_general.iloc[:, :-1], df_general.index.get_level_values(0)   # Features & Target variable (region labels)

<h3>Feature Selection & Visualization General Data</h3>

In [4]:
k_best_feature_general = selectKbest(X_general, y_general, k=5)

Feature 'std_d_per_team_atk': 3.0094731245506234
Feature 'std_fk_per_team': 1.6494232091222396
Feature 'avrg_fk_per_team_atk': 1.563641987680239
Feature 'std_fk_per_team_atk': 1.99837598145269
Feature 'avrg_fk_per_team_dfs': 1.3647796440760942


In [10]:
plot_t_sne(X_general,y_general)

In [13]:
plot_t_sne(X_general[k_best_feature_general],y_general)

In [ ]:
k_best_feature_rfecv = RFECV_feature_selection(X_general,y_general,"linear")

In [4]:
visualize_mean_feature_for_each_region(df_general,k_best_feature_general)

Plots saved as PNG files.


In [ ]:
plot_q_q(X_general)

<h2>Performance data</h2>

In [5]:
performance_feature_emea = performance_feature_creation_for_teams(performance_emea, economy_emea)
performance_feature_americas = performance_feature_creation_for_teams(performance_americas, economy_americas)
performance_feature_pacific = performance_feature_creation_for_teams(performance_pacific, economy_pacific)

df_performance = create_dataframe(performance_feature_emea,performance_feature_americas,performance_feature_pacific)
columns_to_drop = ['5k_mean', '5k_std','1v5_mean', '1v5_std']
df_performance_dropped = df_performance.drop(columns=columns_to_drop)
X_performance, y_performance = df_performance_dropped.iloc[:, :-1], df_performance_dropped.index.get_level_values(0)

In [6]:
k_best_feature_performance = selectKbest(X_performance, y_performance, k=5)

Feature '2k_std': 0.7833969591481704
Feature '4k_mean': 1.1873820438640228
Feature '1v3_std': 0.6534339325305459
Feature 'econ_std': 0.7659081599989729
Feature 'pl_mean': 0.9437653919871462


In [17]:
k_best_feature_rfecv_performance = RFECV_feature_selection(X_performance,y_performance,"linear")

Selected Features:
Index(['2k_mean', '2k_std', '3k_std', '4k_mean', '1v1_mean', '1v1_std',
       '1v2_mean', '1v2_std', '1v3_mean', '1v3_std', 'pl_mean', 'pl_std',
       'de_mean'],
      dtype='object')
Optimal number of features: 13


In [19]:
plot_t_sne(X_performance,y_performance)

In [20]:
plot_t_sne(X_performance[k_best_feature_performance],y_performance)

In [8]:
visualize_mean_feature_for_each_region(df_performance,k_best_feature_performance)

Plots saved as PNG files.


In [7]:
plot_q_q(X_performance[k_best_feature_performance])

<h2>Economic Data</h2>

In [7]:
df_emea_economy = economy_feature_creation_for_teams(economy_emea)
df_americas_economy = economy_feature_creation_for_teams(economy_americas)
df_pacific_economy = economy_feature_creation_for_teams(economy_pacific)

df_economy = create_dataframe(df_emea_economy,df_americas_economy,df_pacific_economy)
# Assuming the last column of the data is the target variable (e.g., region labels)
X_economy, y_economy = df_economy.iloc[:, :-1], df_economy.index.get_level_values(0)   # Features & Target variable (region labels)

In [8]:
k_best_feature_economy = selectKbest(X_economy, y_economy, k=5, selec_type='chi2')

Feature 'ratio_$_won': 0.29450331125827856
Feature 'ratio_Eco': 0.46550512151034384
Feature 'ratio_$': 0.569992553983619
Feature 'ratio_$$$': 0.2927293372377933
Feature 'Bank': 0.28270270849221024


In [45]:
k_best_feature_rfecv_economy= RFECV_feature_selection(X_economy,y_economy,"linear")

Selected Features:
Index(['ratio_Eco'], dtype='object')
Optimal number of features: 1


In [30]:
plot_t_sne(X_economy,y_economy)

In [31]:
plot_t_sne(X_economy[k_best_feature_economy],y_economy)

In [11]:
visualize_mean_feature_for_each_region(df_economy,k_best_feature_economy)

Plots saved as PNG files.


In [ ]:
plot_q_q(X_economy)

<h2>Picks and Bans</h2>

In [9]:
df_emea_pick_bans = picks_and_bans_feature_creation_for_teams(pick_ban_emea)
df_americas_pick_bans = picks_and_bans_feature_creation_for_teams(pick_ban_americas)
df_pacific_pick_bans = picks_and_bans_feature_creation_for_teams(pick_ban_pacific)

df_picks_bans = create_dataframe(df_emea_pick_bans,df_americas_pick_bans,df_pacific_pick_bans)
# Assuming the last column of the data is the target variable (e.g., region labels)
X_picks_bans, y_picks_bans = df_picks_bans.iloc[:, :-1], df_picks_bans.index.get_level_values(0)   # Features & Target variable (region labels)

c:\Users\mariu\OneDrive\Bureau\Valorant\M8_Valorant\utils.py:1490: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  bans_dummies = pd.get_dummies(pick_ban['Bans'].apply(lambda x : ast.literal_eval(x)).apply(pd.Series).stack()).sum(level=0)
c:\Users\mariu\OneDrive\Bureau\Valorant\M8_Valorant\utils.py:1491: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  picks_dummies = pd.get_dummies(pick_ban['Picks'].apply(lambda x : ast.literal_eval(x)).apply(pd.Series).stack()).sum(level=0)
c:\Users\mariu\OneDrive\Bureau\Valorant\M8_Valorant\utils.py:1492: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Us

In [10]:
k_best_feature_picks_bans = selectKbest(X_picks_bans, y_picks_bans, k=5, selec_type='chi2')

Feature 'bans_breeze_mean': 0.9974740036595711
Feature 'picks_lotus_mean': 1.9092904001556068
Feature 'picks_sunset_mean': 1.9605536107869832
Feature 'decider_bind_mean': 1.3287228090959953
Feature 'decider_lotus_mean': 0.9308752146964382


In [37]:
plot_t_sne(X_picks_bans,y_picks_bans)

In [38]:
plot_t_sne(X_picks_bans[k_best_feature_picks_bans],y_picks_bans)

In [14]:
visualize_mean_feature_for_each_region(df_picks_bans,k_best_feature_picks_bans)

Plots saved as PNG files.


In [ ]:
plot_q_q(X_picks_bans)

<h1>Aggregate the data</h1>

In [14]:
df_general_refined = df_general[k_best_feature_general]
df_performance_refined = df_performance[k_best_feature_performance]
df_economy_refined = df_economy[k_best_feature_economy]
df_pick_bans_refined = df_picks_bans[k_best_feature_picks_bans]

In [27]:
df_economy_refined.index

MultiIndex([(    'EMEA',  'FUT'),
            (    'EMEA',   'TL'),
            (    'EMEA',  'KOI'),
            (    'EMEA',  'VIT'),
            (    'EMEA',   'GX'),
            (    'EMEA',   'KC'),
            (    'EMEA',   'TH'),
            (    'EMEA',  'BBL'),
            (    'EMEA',   'M8'),
            (    'EMEA', 'NAVI'),
            (    'EMEA',  'FNC'),
            ('Americas',  'FUR'),
            ('Americas',  'LEV'),
            ('Americas',  'SEN'),
            ('Americas',  'KRÜ'),
            ('Americas', 'MIBR'),
            ('Americas', '100T'),
            ('Americas',   'C9'),
            ('Americas',   'G2'),
            ('Americas',   'EG'),
            ('Americas',  'NRG'),
            ('Americas', 'LOUD'),
            ( 'Pacific',  'TLN'),
            ( 'Pacific',   'T1'),
            ( 'Pacific',  'RRQ'),
            ( 'Pacific', 'ZETA'),
            ( 'Pacific',   'TS'),
            ( 'Pacific',  'DRX'),
            ( 'Pacific',  'GEN'),
            ( 

In [31]:
df_general_refined.index.names = ['Region', 'Team Name']
df_performance_refined.index.names = ['Region', 'Team Name']
df_economy_refined.index.names = ['Region', 'Team Name']
df_pick_bans_refined.index.names = ['Region', 'Team Name']

In [ ]:
merged_df.index = merged_df.index.set_levels(merged_df.index.levels[1].map(lambda x: your_transformation(x)), level=1)

In [22]:
merged_df = pd.concat([df_performance_refined, df_economy_refined, df_pick_bans_refined], axis=1)

In [23]:
merged_df

2k_std   4k_mean   1v3_std  econ_std   pl_mean  ratio_$_won  \
EMEA     BBL   0.600763  0.214091  0.000000  0.242057  0.288372         0.20   
         FNC   0.170427  0.490626  0.797940  0.272316  0.511628         1.00   
         FUT   0.810584  0.252910  0.721798  0.344514  1.000000         1.00   
         GX    0.267792  0.929456  0.566280  0.722095  0.604651         0.00   
         KC    0.571292  0.566846  0.217722  0.455306  0.573643         0.40   
         KOI   0.493153  0.629206  0.288966  0.227375  0.410148         0.12   
         M8    0.905667  0.576400  0.000000  0.281281  0.623256         0.50   
         NAVI  0.362891  0.896936  0.480950  0.538118  0.112957         0.00   
         TH    0.585511  0.765103  0.675180  1.000000  0.455814         0.36   
         TL    0.716810  0.680186  0.537500  0.776990  0.392027         0.28   
         VIT   0.785995  0.778910  0.349593  0.260640  0.968288         0.16   
Americas 100T  0.292853  0.802843  0.716667  0.202338  0.139535         0.44   
         C9    0.485182  0.427302  0.433449  0.000000  0.511628         0.00   
         EG    0.872376  0.713638  0.638352  0.641458  0.567442         0.66   
         FUR   0.555713  0.039647  0.000000  0.247230  0.162791         0.00   
         G2    0.478097  0.586203  0.227983  0.392797  0.618962         0.58   
         KRÜ   0.773503  0.000000  0.638352  0.773114  0.511628         0.40   
         LEV   0.298607  0.982212  0.000000  0.433443  0.604651         0.20   
         LOUD  0.213276  0.328511  0.000000  0.277726  0.490161         0.58   
         MIBR  0.036380  0.343930  0.266991  0.175789  0.720930         0.72   
         NRG   0.455874  0.321592  0.000000  0.447354  0.511628         0.76   
         SEN   0.360697  0.325973  0.366855  0.441619  0.352159         0.46   
Pacific  BLD   0.767797  0.143313  0.575563  0.358456  0.790698         0.34   
         DFM   0.678734  0.277987  1.000000  0.100164  0.093023         0.00   
         DRX   0.314346  0.411818  0.527960  0.058717  0.651163         0.50   
         GE    0.000000  0.480149  0.000000  0.199045  0.591362         0.00   
         GEN   0.498437  1.000000  0.324336  0.538020  0.386047         0.76   
         PRX   0.283852  0.540833  0.367637  0.309856  0.356589         0.00   
         RRQ   0.412729  0.398798  0.516314  0.234789  0.465116         0.34   
         T1    0.440158  0.784783  0.000000  0.315668  0.431894         0.00   
         TLN   0.521218  0.625896  0.575563  0.844921  0.000000         0.80   
         TS    0.611177  0.333548  0.305299  0.411533  0.536998         0.36   
         ZETA  1.000000  0.751688  0.332790  0.492505  0.131078         0.16   

               ratio_Eco   ratio_$  ratio_$$$      Bank  bans_breeze_mean  \
EMEA     BBL    0.634615  0.886076   0.268966  0.000000          0.500000   
         FNC    0.365385  0.189873   0.593103  0.536060          0.000000   
         FUT    0.288462  0.316456   0.993103  0.742463          1.000000   
         GX     1.000000  0.443038   0.924138  0.269611          0.000000   
         KC     0.461538  0.278481   1.000000  0.524424          0.857143   
         KOI    0.250000  0.468354   0.331034  0.412498          0.000000   
         M8     0.711538  0.101266   0.068966  0.346620          0.500000   
         NAVI   0.115385  0.139241   0.151724  0.771793          0.000000   
         TH     0.423077  0.291139   0.565517  0.675738          0.000000   
         TL     0.403846  0.734177   0.931034  0.750914          0.666667   
         VIT    0.326923  0.253165   0.000000  0.735745          0.200000   
Americas 100T   0.480769  0.569620   0.662069  0.070685          1.000000   
         C9     0.000000  0.240506   0.241379  0.414790          1.000000   
         EG     0.538462  1.000000   0.827586  0.169978          0.000000   
         FUR    0.307692  0.468354   0.737931  0.365852          0.000000   
         G2     0.057692  0.341772   0.703448  0.694711         

In [ ]:
merged_df = pd.merge(df_general_refined, df_performance_refined, on="team name", suffixes=('_df1', '_df2'))
merged_df = pd.merge(merged_df, df3, on="team name")
merged_df = pd.merge(merged_df, df4, on="team name", suffixes=('_df3', '_df4'))

<h1>Resample & Boostrap my data</h1>

In [ ]:
import numpy as np

# Assuming your original dataset is stored in X and y
# X is the feature matrix and y is the target variable

# Number of bootstrap datasets to create for each class
num_bootstrap_datasets = 10

# Size of each bootstrap dataset (same as original class size)
bootstrap_dataset_size = 11

# List to store augmented data
augmented_X = []
augmented_y = []

# Create bootstrap datasets for each class
for class_label in np.unique(y):
    # Indices of samples belonging to the current class
    class_indices = np.where(y == class_label)[0]
    
    # Create bootstrap datasets for the current class
    for _ in range(num_bootstrap_datasets):
        # Randomly sample indices with replacement
        bootstrap_indices = np.random.choice(class_indices, size=bootstrap_dataset_size, replace=True)
        
        # Create bootstrap dataset for the current class
        bootstrap_X = X[bootstrap_indices]
        bootstrap_y = y[bootstrap_indices]
        
        # Append bootstrap dataset to augmented data
        augmented_X.append(bootstrap_X)
        augmented_y.append(bootstrap_y)

# Convert augmented data to numpy arrays
augmented_X = np.vstack(augmented_X)
augmented_y = np.concatenate(augmented_y)

<h1>Centralization of the Analysis</h1>

<h2>Selected Feature for the Individuality</h2>
<p>K1 best from general</p>
<p>K2 best from performance</p>

<h2>Selected Feature for the Overall Strategy</h2>
<p>K3 best from economy</p>
<p>K4 best from picks and bans</p>

<h2>Selected Feature for the State of Form</h2>
<p>Not done yet</p>